In [1]:
#importing
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, Activation, GlobalAveragePooling2D

Accessing Img

In [2]:
#opening img 
img = cv2.imread("E:/3rd_Year_CWIT/FYP/sign-dataset/sample_Signature/sample_Signature/genuine/NFI-01301013.png")
img_g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
plt.imshow(img_g)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [3]:
#program to show image with key_points and Descriptors
#creating sift obj
sift = cv2.SIFT_create(nfeatures=1000, nOctaveLayers=4, contrastThreshold=0.04, edgeThreshold=15, sigma=1.8)

#detect keypoints and descriptors
keypoints , descriptors = sift.detectAndCompute(image = img_g, mask = None, descriptors= None)

# Draw the keypoints on the image
img_g_with_keypoints = cv2.drawKeypoints(img_g, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Draw the descriptors on the image
img_g_with_descriptors = cv2.drawKeypoints(img_g, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display the images
cv2.imshow('Image with keypoints', img_g_with_keypoints)
cv2.imshow('Image with descriptors', img_g_with_descriptors)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'img_g' is not defined

# Signature Matching (SIFT)

In [1]:

#GOAL : feature matching
img1 = cv2.imread(r"D:\final_year_project\data\genuine\005005_000.png")
img1 = cv2.cvtColor(img1 , cv2.COLOR_RGB2GRAY) #RGB->GRAY
img1 = cv2.resize(img1, (800,600))
img2 = cv2.imread(r"D:\final_year_project\data\genuine\005005_000.png")
img2 = cv2.cvtColor(img2 , cv2.COLOR_RGB2GRAY) #RGB->GRAY
img2 = cv2.resize(img2, (800,600))
#sift obj
#sift = cv2.SIFT_create(nfeatures=1000, nOctaveLayers=5, contrastThreshold=0.10, edgeThreshold=11, sigma=1.8)
sift = cv2.SIFT_create(nfeatures=500, nOctaveLayers=5, contrastThreshold=0.09, edgeThreshold=11, sigma=1.8)


#detect keypoints and descriptors
kp1 , des1 = sift.detectAndCompute(image = img1, mask = None, descriptors= None)
kp2 , des2 = sift.detectAndCompute(image = img2, mask = None, descriptors= None)

#BFMatcher with def params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2,k=3)

print(f"Length of Matches :- {len(matches)}")
# Apply ratio test
good = []
for match_list in matches:
    m, n = match_list[:2]  # Extract the first two matches
    if m.distance < 0.75 * n.distance:
        good.append([m])
        
# Calculate the percentage of match
percentage_match = len(good) / len(kp1) * 100
print(f"Percentage of match: {percentage_match:.2f}%")
if percentage_match < 13.5:
    print("forged")
else:
    print("orginal")

print(f"Length of good :- {len(good)}")
# Draw the matches
img_matches = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the image
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [5]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Load and preprocess the images
img1_path = r"D:\final_year_project\data\genuine\005005_000.png"
img2_path = r"D:\final_year_project\data\genuine\005005_000.png"

img1 = cv2.imread(img1_path, 0)
img2 = cv2.imread(img2_path, 0)

# Check if the images were loaded successfully
if img1 is None or img2 is None:
    print("Failed to load images.")
    exit()

# Resize and normalize the images
img1 = cv2.resize(img1, (800, 600))
img2 = cv2.resize(img2, (800, 600))
img1 = cv2.bitwise_not(img1)
img2 = cv2.bitwise_not(img2)

# Extract SIFT features
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Extract geometric features
def extract_geometric_features(image):
    contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return np.zeros(6)
    
    c = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(c)
    (x, y), (w, h), angle = rect
    
    aspect_ratio = float(w) / h
    rect_area = w * h
    hull_area = cv2.contourArea(c)
    solidity = float(rect_area) / hull_area
    extent = float(cv2.contourArea(c)) / rect_area
    equiv_diameter = np.sqrt(4 * cv2.contourArea(c) / np.pi)
    
    return [aspect_ratio, solidity, extent, equiv_diameter, w, h]

geom_features1 = extract_geometric_features(img1)
geom_features2 = extract_geometric_features(img2)

# Combine SIFT and geometric features
features1 = np.concatenate((des1.flatten(), geom_features1))
features2 = np.concatenate((des2.flatten(), geom_features2))

# Normalize the features
scaler1 = StandardScaler()
scaler2 = StandardScaler()
features1 = scaler1.fit_transform(features1.reshape(1, -1))
features2 = scaler2.fit_transform(features2.reshape(1, -1))

# Train an SVM classifier
X_train = np.concatenate((features1, features2), axis=0)
y_train = np.array([1, 0])  # Assuming the first signature is genuine and the second is forged

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train, y_train)

# Test the classifier on a new signature
#test_img_path = r"D:\final_year_project\data\forged\021005_002.png"
test_img_path = r"D:\final_year_project\data\forged\021006_000.png"
test_img = cv2.imread(test_img_path, 0)

# Check if the test image was loaded successfully
if test_img is None:
    print("Failed to load the test image.")
    exit()

test_img = cv2.resize(test_img, (800, 600))
test_img = cv2.bitwise_not(test_img)

_, test_des = sift.detectAndCompute(test_img, None)
test_geom_features = extract_geometric_features(test_img)
test_features = np.concatenate((test_des.flatten(), test_geom_features))

# Pad or truncate the test features to match the expected number of features
expected_features = scaler1.n_features_in_
test_feature_size = test_features.size
if test_feature_size < expected_features:
    test_features = np.pad(test_features, (0, expected_features - test_feature_size), mode='constant')
elif test_feature_size > expected_features:
    test_features = test_features[:expected_features]

test_features = scaler1.transform(test_features.reshape(1, -1))  # Apply scaling from scaler1

prediction = clf.predict(test_features)

if prediction[0] == 1:
    print("Genuine signature")
else:
    print("Forged signature")

Genuine signature


In [ ]:
import cv2
import numpy as np
from dtaidistance import dtw

# Load and preprocess the images
img1_path = r"D:\final_year_project\data\genuine\005005_000.png"
img2_path = r"D:\final_year_project\data\genuine\005005_001.png"

img1 = cv2.imread(img1_path, 0)
img2 = cv2.imread(img2_path, 0)

# Check if the images were loaded successfully
if img1 is None or img2 is None:
    print("Failed to load images.")
    exit()

# Resize and normalize the images
img1 = cv2.resize(img1, (800, 600))
img2 = cv2.resize(img2, (800, 600))
img1 = cv2.bitwise_not(img1)
img2 = cv2.bitwise_not(img2)

# Extract pen coordinates from the images
def extract_pen_coordinates(image):
    coords = np.where(image > 0)
    return np.ravel(np.column_stack(coords))


coords1 = extract_pen_coordinates(img1)
coords2 = extract_pen_coordinates(img2)

# Compute DTW distance between the signatures
distance = dtw.distance(coords1, coords2)

# Set a threshold for classifying as genuine or forged
threshold = 50  # Adjust this value based on your data

if distance < threshold:
    print("Genuine signature")
else:
    print("Forged signature")

# Optionally, you can visualize the warping path
warping_path = dtw.warping_path(coords1, coords2)
warped_coords1 = coords1[warping_path[0]]
warped_coords2 = coords2[warping_path[1]]

# Display the warped signatures (optional)
warped_img1 = np.zeros_like(img1)
warped_img2 = np.zeros_like(img2)

for x, y in warped_coords1:
    warped_img1[y, x] = 255

for x, y in warped_coords2:
    warped_img2[y, x] = 255

cv2.imshow('Warped Signature 1', warped_img1)
cv2.imshow('Warped Signature 2', warped_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
pip install dtaidistance

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2

class FeatureMatching:
    def __init__(self, image1_path, image2_path):
        self.img1 = cv2.imread(image1_path)
        self.img2 = cv2.imread(image2_path)

    def preprocess_images(self):
        self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_BGR2GRAY)
        self.img1 = cv2.resize(self.img1, (800, 600))
        self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_BGR2GRAY)
        self.img2 = cv2.resize(self.img2, (800, 600))

    def find_features(self):
        self.sift = cv2.SIFT_create(nfeatures=500, nOctaveLayers=5, contrastThreshold=0.09, edgeThreshold=11, sigma=1.8)
        self.kp1, self.des1 = self.sift.detectAndCompute(self.img1, None)
        self.kp2, self.des2 = self.sift.detectAndCompute(self.img2, None)

    def match_features(self):
        bf = cv2.BFMatcher()
        self.matches = bf.knnMatch(self.des1, self.des2, k=3)

    def filter_matches(self):
        self.good_matches = []
        for match_list in self.matches:
            m, n = match_list[:2]
            if m.distance < 0.75 * n.distance:
                self.good_matches.append([m])

    def calculate_match_percentage(self):
        self.match_percentage = len(self.good_matches) / len(self.kp1) * 100

    def check_authenticity(self):
        if self.match_percentage < 13.5:
            print("Forged")
        else:
            print("Original")

    def draw_matches(self):
        img_matches = cv2.drawMatchesKnn(self.img1, self.kp1, self.img2, self.kp2, self.good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        cv2.imshow('Matches', img_matches)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    image1_path = r"D:\final_year_project\data\genuine\005005_000.png"
    image2_path = r"D:\final_year_project\data\genuine\005005_000.png"

    feature_matching = FeatureMatching(image1_path, image2_path)
    feature_matching.preprocess_images()
    feature_matching.find_features()
    feature_matching.match_features()
    feature_matching.filter_matches()
    feature_matching.calculate_match_percentage()
    print(f"Length of Matches: {len(feature_matching.matches)}")
    print(f"Percentage of match: {feature_matching.match_percentage:.2f}%")
    print(f"Length of good matches: {len(feature_matching.good_matches)}")
    feature_matching.check_authenticity()
    feature_matching.draw_matches()

Length of Matches: 65
Percentage of match: 100.00%
Length of good matches: 65
Original
